### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from neuropy.analyses import Pf1D
import subjects

### Example plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d
import subjects

sessions = subjects.sd.ratUday4

In [ ]:
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.lin_maze
    # pos.t_start = pos.t_start +0.5
    # run = sess.run['backward']
    pf1d = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=4,
        sigma=4,
        grid_bin=2,
        # epochs=run,
        frate_thresh=3,
    )

    pf_neurons = neurons.get_by_id(pf1d.neuron_ids)

In [ ]:
from neuropy.plotting import plot_ratemap

plot_ratemap(pf1d,normalize_tuning_curve=True)

### Test placefields

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from neuropy.core import Position,Neurons
from neuropy.analyses import Pf1D
from neuropy import plotting

t= np.linspace(0,1,240000)
y = np.sin(2*np.pi*12*t)*100

pos = Position(traces=y.reshape(1,-1),t_start=0,sampling_rate=120)

spktrns = []
for i in range(-100,100,30):
    indices = np.where((pos.x>=i)&(pos.x<=i+20))[0]
    indices = np.random.choice(indices,4000)
    spktrns.append(indices/120)
spktrns = np.array(spktrns)

neurons = Neurons(spiketrains=spktrns,t_start=0,t_stop=2000)
pf1d = Pf1D(neurons=neurons,position=pos,speed_thresh=0.1,grid_bin=5,sigma=7)
# _,ax = plt.subplots()

# ax.plot(pos.time,pos.x)
# plotting.plot_raster(neurons,ax=ax)


In [ ]:
%matplotlib widget

# plotting.plot_ratemap(pf1d.ratemap,normalize_tuning_curve=False)
plt.plot(pf1d.occupancy)

### Pool placefields into one file

In [ ]:
%matplotlib widget
import numpy as np
import pandas as pd
from neuropy.analyses import Pf1D
from neuropy import plotting
import subjects

sessions = subjects.sd.pf_sess + subjects.nsd.pf_sess 

In [ ]:
from neuropy.utils import mathutil

norm_tuning_all = pd.DataFrame()
for sub, sess in enumerate(sessions):

    maze = sess.paradigm["maze"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.maze
    # pos.t_start = pos.t_start +0.5
    # run = sess.run['backward']
    pf1d = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=3,
        sigma=4,
        grid_bin=4,
        # epochs=run,
        frate_thresh=1,
    )

    norm_tuning = mathutil.min_max_scaler(pf1d.tuning_curves, axis=-1)
    x_norm = mathutil.min_max_scaler(pf1d.xbin_centers)
    x_req = np.linspace(0, 1, 21).round(decimals=2)
    interp = lambda y: np.interp(x_req, x_norm, y)

    tuning_x_req = np.apply_along_axis(interp, axis=1, arr=norm_tuning)

    df = pd.DataFrame(tuning_x_req,columns=x_req)
    df['grp']= sess.tag

    norm_tuning_all = norm_tuning_all.append(df,ignore_index=True)


subjects.GroupData().save(norm_tuning_all,'pf_norm_tuning')


In [ ]:
plotting.plot_ratemap(pf1d,normalize_tuning_curve=True)

### Place field similarity between maze and remaze

In [ ]:
sessions = subjects.nsd.ratUday2

In [ ]:
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()
    neurons = sess.neurons.get_neuron_type(['pyr'])
    good_cells = neurons.firing_rate<10
    neurons = neurons[good_cells]
    
    maze_pos = sess.maze
    maze_run = sess.maze_run
    pf1 = Pf1D(
        neurons=neurons,
        position=maze_pos,
        sigma=4,
        grid_bin=2,
        epochs=maze_run["up"],
        frate_thresh=0,
    )

    remaze_pos = sess.remaze
    remaze_run = sess.remaze_run
    pf2 = Pf1D(
        neurons=neurons,
        position=remaze_pos,
        sigma=4,
        grid_bin=2,
        epochs=remaze_run["up"],
        frate_thresh=0,
    )




In [ ]:
np.corrcoef(pf1.tuning_curves[:,:114].flatten(),pf2.tuning_curves[:,:114].flatten())

In [ ]:
from neuropy.plotting import plot_ratemap
_,axs = plt.subplots(1,2)

plot_ratemap(pf1,normalize_tuning_curve=True,ax=axs[0])
plot_ratemap(pf2,normalize_tuning_curve=True,ax=axs[1])